### EDA for Superstore Regional Sales Dashboard

#### Gather useful metrics from the Superstore sales data and place in a Plotly powered dashboard

In [68]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [71]:
# Read in the data
df = pd.read_csv("../data/Superstore.csv", index_col="Row ID", parse_dates=["Order Date", "Ship Date"])
df.head(2)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,"42,420.00",South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
2,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,"42,420.00",South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.00,219.58


In [72]:

# Replace NaNs with 0
df["Postal Code"].fillna(0, inplace=True) 

# Convert Postal Code to String
df["Postal Code"] = df["Postal Code"].apply(lambda pc: str(round(pc)))

In [73]:
# Create new Order year column
df["Order Year"] = df["Order Date"].dt.year
# Create new Order month column
df["Order Month"] = df["Order Date"].dt.month

In [146]:
# Save the processed file
df.to_csv('../data/prepped_superstore.csv', index=True, header=True)

In [147]:
# Some useful varables
total_sales = df["Sales"].sum()
total_profit = df["Profit"].sum()
total_profit_ratio = round(total_profit/total_sales, 2)
total_items_sold = df["Quantity"].sum()

In [148]:
# Some useful functions

def get_regional_data(region_name):
    return df[df["Region"] == region_name]

def get_regional_percent_of_total_sales(region_name, pretty=True):
    regional_sales_total = get_regional_data(region_name=region_name)["Sales"].sum()
    percentage = round(regional_sales_total/total_sales, 2)
    if (pretty):
        percentage = get_pretty_percent(regional_sales_total, total_sales)
    return percentage

def get_regional_percent_of_total_profit(region_name, pretty=True):
    regional_profit_total = get_regional_data(region_name=region_name)["Profit"].sum()
    percentage = round(regional_profit_total/total_profit,2)
    if (pretty):
        percentage = get_pretty_percent(regional_profit_total, total_profit)
    return percentage

def get_regional_profit_ratio(region_name, pretty=True):
    regional_data = get_regional_data(region_name=region_name)
    regional_profit_total = regional_data["Profit"].sum()
    regional_sales_total = regional_data["Sales"].sum()
    regional_profit_ratio = round(regional_profit_total / regional_sales_total, 2)
    if (pretty):
        regional_profit_ratio = get_pretty_percent(regional_profit_total, regional_sales_total)
    return regional_profit_ratio

def get_pretty_percent(x, y):
    percentage = round(x/y, 2)
    return "{:.0%}".format(percentage) 

In [149]:
west = get_regional_data("West")
west.shape[0]

3203

In [150]:
print(west["Order Year"].min())
print(west["Order Year"].max())
print(west["Sales"].sum())
print(get_regional_percent_of_total_sales("West"))
print(west["Profit"].sum())
print(get_regional_percent_of_total_profit("West"))
print(get_regional_profit_ratio("West"))



2016
2019
725457.8245
32%
108418.44889999999
38%
15%


In [159]:
print("ALL")
avg_order_sale_by_year = df.groupby("Order Year")["Sales"].mean()
print(avg_order_sale_by_year)
print("WEST")
west_avg_order_sale_by_year = get_regional_data("West").groupby("Order Year")["Sales"].mean()
print(west_avg_order_sale_by_year)

print("ALL")
avg_order_profit_by_year = df.groupby("Order Year")["Profit"].mean()
print(avg_order_profit_by_year)
print("WEST")
west_avg_order_profit_by_year = get_regional_data("West").groupby("Order Year")["Profit"].mean()
print(west_avg_order_profit_by_year)

ALL
Order Year
2016   242.97
2017   223.85
2018   235.49
2019   221.38
Name: Sales, dtype: float64
WEST
Order Year
2016   223.73
2017   218.02
2018   232.89
2019   228.43
Name: Sales, dtype: float64
ALL
Order Year
2016   24.86
2017   29.31
2018   31.62
2019   28.21
Name: Profit, dtype: float64
WEST
Order Year
2016   30.36
2017   31.92
2018   29.88
2019   40.01
Name: Profit, dtype: float64


In [151]:
def get_sales_by_group(groupby_col, region_name=None):
    grouped_sales = pd.Series()
    if (region_name):
        region_df = get_regional_data(region_name)
        grouped_sales = region_df.groupby([groupby_col])["Sales"].sum()
    else:
        grouped_sales = df.groupby([groupby_col])["Sales"].sum()
    return grouped_sales


def get_profit_by_group(groupby_col, region_name=None):
    grouped_profit = pd.Series()
    if (region_name):
        region_df = get_regional_data(region_name)
        grouped_profit = region_df.groupby([groupby_col])["Profit"].sum()
    else:
        grouped_profit = df.groupby([groupby_col])["Profit"].sum()
    return grouped_profit


In [152]:
sales_by_cat = get_sales_by_group("Category")
profit_by_cat = get_profit_by_group("Category")

profit_ratio_by_cat_df = pd.concat([sales_by_cat, profit_by_cat], axis=1)
profit_ratio_by_cat_df["Profit Ratio"] = profit_ratio_by_cat_df.apply(lambda row : round(row["Profit"]/row["Sales"], 2), axis=1)

print(sales_by_cat)
print(profit_by_cat)
print(profit_ratio_by_cat_df)


sales_by_subcat = get_sales_by_group("Sub-Category")
profit_by_subcat = get_profit_by_group("Sub-Category")

profit_ratio_by_subcat_df = pd.concat([sales_by_subcat, profit_by_subcat], axis=1)
profit_ratio_by_subcat_df["Profit Ratio"] = profit_ratio_by_subcat_df.apply(lambda row: round(row["Profit"]/row["Sales"],2), axis=1)


print(sales_by_subcat)
print(profit_by_subcat)
print(profit_ratio_by_subcat_df)

Category
Furniture         741,999.80
Office Supplies   719,047.03
Technology        836,154.03
Name: Sales, dtype: float64
Category
Furniture          18,451.27
Office Supplies   122,490.80
Technology        145,454.95
Name: Profit, dtype: float64
                     Sales     Profit  Profit Ratio
Category                                           
Furniture       741,999.80  18,451.27          0.02
Office Supplies 719,047.03 122,490.80          0.17
Technology      836,154.03 145,454.95          0.17
Sub-Category
Accessories   167,380.32
Appliances    107,532.16
Art            27,118.79
Binders       203,412.73
Bookcases     114,880.00
Chairs        328,449.10
Copiers       149,528.03
Envelopes      16,476.40
Fasteners       3,024.28
Furnishings    91,705.16
Labels         12,486.31
Machines      189,238.63
Paper          78,479.21
Phones        330,007.05
Storage       223,843.61
Supplies       46,673.54
Tables        206,965.53
Name: Sales, dtype: float64
Sub-Category
Accessories 

In [153]:
west_sales_by_cat = get_sales_by_group(groupby_col="Category", region_name="West")
west_profit_by_cat = get_profit_by_group(groupby_col="Category", region_name="West")

west_profit_ratio_by_cat_df = pd.concat([west_sales_by_cat, west_profit_by_cat], axis=1)
west_profit_ratio_by_cat_df["Profit Ratio"] = west_profit_ratio_by_cat_df.apply(lambda row : round(row["Profit"]/row["Sales"], 2), axis=1)

print(west_sales_by_cat)
print(west_profit_by_cat)


west_sales_by_subcat = get_sales_by_group(groupby_col="Sub-Category", region_name="West")
west_profit_by_subcat = get_profit_by_group(groupby_col="Sub-Category", region_name="West")
west_profit_ratio_by_subcat_df = pd.concat([west_sales_by_subcat, west_profit_by_subcat], axis=1)
west_profit_ratio_by_subcat_df["Profit Ratio"] = profit_ratio_by_subcat_df.apply(lambda row: round(row["Profit"]/row["Sales"],2), axis=1)


print(west_sales_by_subcat)
print(west_profit_by_subcat)
print(west_profit_ratio_by_subcat_df)


Category
Furniture         252,612.74
Office Supplies   220,853.25
Technology        251,991.83
Name: Sales, dtype: float64
Category
Furniture         11,504.95
Office Supplies   52,609.85
Technology        44,303.65
Name: Profit, dtype: float64
Sub-Category
Accessories    61,114.12
Appliances     30,236.34
Art             9,212.07
Binders        55,961.11
Bookcases      36,004.12
Chairs        101,781.33
Copiers        49,749.24
Envelopes       4,118.10
Fasteners         923.22
Furnishings    30,072.73
Labels          5,078.73
Machines       42,444.12
Paper          26,663.72
Phones         98,684.35
Storage        70,532.85
Supplies       18,127.12
Tables         84,754.56
Name: Sales, dtype: float64
Sub-Category
Accessories   16,484.60
Appliances     8,261.27
Art            2,374.10
Binders       16,096.80
Bookcases     -1,646.51
Chairs         4,027.58
Copiers       19,327.24
Envelopes      1,908.76
Fasteners        275.19
Furnishings    7,641.27
Labels         2,303.12
Machines   